In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
print(keras.backend.backend())

2.16.1
3.3.3
tensorflow


# Import Nasdaq-100 List

In [19]:
ndx100_df = pd.read_csv(
    "ndx_100.csv", 
    usecols=["Company", "Symbol", "Portfolio%"])

display(ndx100_df)

,Company,Symbol,Portfolio%
0,Microsoft Corp,MSFT,8.71%
1,Apple Inc,AAPL,7.54%
2,NVIDIA Corp,NVDA,6.33%
3,Amazon.com Inc,AMZN,5.38%
4,Broadcom Inc,AVGO,4.64%
...,...,...,...
96,Take-Two Interactive Software Inc,TTWO,0.18%
97,Warner Bros Discovery Inc,WBD,0.15%
98,Illumina Inc,ILMN,0.15%
99,Walgreens Boots Alliance Inc,WBA,0.11%


## Market Cap and Outstanding shares:

From [https://www.nasdaq.com/market-activity/quotes/nasdaq-ndx-index](https://www.nasdaq.com/market-activity/quotes/nasdaq-ndx-index):

In [41]:
ndx_mktcap_df = pd.read_csv(
    "nsdq100_marketcap.csv", 
    usecols=["Symbol", "Name", "Market Cap", "Last Sale"])

# Check all the symbols are listed:
print("All Symbols: ", set(ndx100_df["Symbol"].to_list()) == set(ndx_mktcap_df["Symbol"].to_list()))

# Compute the number of outstanding shares:
ndx_mktcap_df["Market Cap"] = ndx_mktcap_df["Market Cap"].apply(lambda s: float(s.replace(",", "")))
ndx_mktcap_df["Last Sale"] = ndx_mktcap_df["Last Sale"].apply(lambda s: float(s.replace("$", "").replace(",", "")))
ndx_mktcap_df["Num Outstanding Shares"] = ndx_mktcap_df["Market Cap"] / ndx_mktcap_df["Last Sale"]
ndx_mktcap_df["Num Outstanding Shares"] = ndx_mktcap_df["Num Outstanding Shares"].round().astype("int")
ndx_mktcap_df.drop(["Market Cap", "Last Sale"], axis=1, inplace=True)

display(ndx_mktcap_df)
display(ndx_mktcap_df.dtypes)

All Symbols:  True


,Symbol,Name,Num Outstanding Shares
0,AAPL,Apple Inc. Common Stock,15441881000
1,ABNB,"Airbnb, Inc. Class A Common Stock",647064669
2,ADBE,Adobe Inc. Common Stock,448000000
3,ADI,"Analog Devices, Inc. Common Stock",495908150
4,ADP,"Automatic Data Processing, Inc. Common Stock",410790567
...,...,...,...
96,WBA,"Walgreens Boots Alliance, Inc. Common Stock",862713366
97,WBD,"Warner Bros. Discovery, Inc. Series A Common S...",2439687237
98,WDAY,"Workday, Inc. Class A Common Stock",264000000
99,XEL,Xcel Energy Inc. Common Stock,555155770


Symbol                    object
Name                      object
Num Outstanding Shares     int64
dtype: object

# Individual Stocks

In [44]:
import glob

csv_files_list = glob.glob("CSVs/*.csv")
print(len(csv_files_list))

50


In [52]:
ndx_df_dict = {}
for csv_file in csv_files_list:
    symbol = csv_file.split("/")[-1].split(".")[0]
    ndx_df_dict[symbol] = pd.read_csv(csv_file)
    ndx_df_dict[symbol] = ndx_df_dict[symbol].iloc[::-1]
    ndx_df_dict[symbol]["DATETIME"] = pd.to_datetime(ndx_df_dict[symbol]["Date"], format="%m/%d/%Y")
    ndx_df_dict[symbol].reset_index(inplace=True)
    ndx_df_dict[symbol].drop(["index"], axis=1, inplace=True)
    print(f"\t{symbol:>6}\t{len(ndx_df_dict[symbol].index):>6}")

display(ndx_df_dict["MSFT"])

	  CSCO	  2518
	  ISRG	  2518
	   PDD	  1448
	  VRTX	  2518
	  GILD	  2518
	  CDNS	  2518
	   ADI	  2518
	  ADBE	  2518
	   PEP	  2518
	  COST	  2518
	  MDLZ	  2518
	  BKNG	  2518
	  ABNB	   849
	  SBUX	  2518
	  INTU	  2518
	  AMZN	  2518
	  INTC	  2518
	   TXN	  2518
	  MELI	  2518
	  PYPL	  2219
	  MSFT	  2518
	  CTAS	  2518
	  AMAT	  2518
	   AMD	  2518
	  KLAC	  2518
	  NXPI	  2518
	  NVDA	  2518
	 CMCSA	  2518
	   MAR	  2518
	   LIN	   291
	  PANW	  2518
	  NFLX	  2518
	  TSLA	  2518
	 GOOGL	  2518
	  META	  2518
	  CRWD	  1228
	  SNPS	  2518
	  GOOG	  2518
	  AVGO	  2518
	  REGN	  2518
	  AMGN	  2518
	   ADP	  2518
	  QCOM	  2518
	   CSX	  2518
	   HON	  2518
	  AAPL	  2518
	  TMUS	  2518
	    MU	  2518
	  ASML	  2518
	  LRCX	  2518


,Date,Close/Last,Volume,Open,High,Low,DATETIME
0,04/28/2014,$40.87,50598770,$40.14,$41.29,$40.09,2014-04-28
1,04/29/2014,$40.51,29625310,$41.10,$41.19,$40.39,2014-04-29
2,04/30/2014,$40.40,35421760,$40.40,$40.50,$40.1699,2014-04-30
3,05/01/2014,$40.00,28787400,$40.24,$40.36,$39.95,2014-05-01
4,05/02/2014,$39.69,43404480,$40.31,$40.34,$39.66,2014-05-02
...,...,...,...,...,...,...,...
2513,04/22/2024,$400.96,20286880,$400.08,$402.85,$395.75,2024-04-22
2514,04/23/2024,$407.57,15734500,$404.235,$408.20,$403.06,2024-04-23
2515,04/24/2024,$409.06,15065330,$409.56,$412.47,$406.78,2024-04-24
2516,04/25/2024,$399.04,40586450,$394.03,$399.8899,$388.03,2024-04-25
